Theo DeGuzman Johns Hopkins University/ IQT24 Senior Design Projec// V2 Prototype Co
// Not for production use -- end user assumes responsibility for usage and testing of this 


Run cells 1-15 first

Then run bottom cells, copying relevant values into appropriate cells code


In [1]:
#consts
import binascii


START = int("FF", 16)
END = int("FE", 16)

#define VEHICLE_STATE_VECTOR 0xAB
#define VEHICLE_STATE_VECTOR_LENGTH 35
#define VEHICLE_BATTERY_VECTOR 0xAB
#define VEHICLE_BATTERY_VECTOR_LENGTH 35

#define CONTROLLER_STATE 0xAC
#define CONTROLLER_STATE_LENGTH 19

#define WAYPOINT_LIST 0xAD

#define WAYPOINT 0xAE
#define WAYPOINT_LENGTH 23

#define MODE_CHANGE 0xAF
#define MODE_CHANGE_LENGTH 11

#define WAYPOINT_REACHED 0xBA
#define WAYPOINT_REACHED_LENGTH 27

#define MISSION_PARAMS 0xCB
#define MISSION_PARAMS_LEN 59


#define REQUEST_WAYPOINT_LIST 0xBB
REQUEST_WAYPOINT_LIST = int("BB", 16)

#define REQUEST_WAYPOINT 0xBC
REQUEST_WAYPOINT = int("BC", 16)


#define CHANGE_WAYPOINT 0xBD
CHANGE_WAYPOINT = int("BD", 16)

#define GET_MISSION_PARAMS 0xBE
GET_MISSION_PARAMS = int("BE", 16)

#define CHANGE_MISSIONS_PARAMS 0xBF
CHANGE_MISSIONS_PARAMS = int("BF", 16)

#define FORCE_MODE_CHANGE 0xCA
FORCE_MODE_CHANGE = int("CA", 16)



In [2]:
def byteStrToInt(input_string, sign=True):
    if len(input_string) != 8:
        raise ValueError("Input string must be exactly 8 characters long.")
    
    bytes_obj = [(int(input_string[0:2], 16)), (int(input_string[2:4], 16)), (int(input_string[4:6], 16)), (int(input_string[6:8], 16))]
    final_int = int.from_bytes(bytes_obj, 'little', signed=sign)
    return final_int
    
# Example usage
input_string = "60746517"  # Example 8-character string
result = byteStrToInt(input_string)
print("The four-byte integer is:", result)

The four-byte integer is: 392524896


In [3]:
def unpackStateVector(string):
    lat = byteStrToInt(string[4:12])/1E7
    lon = byteStrToInt(string[12:20])/1E7
    speed = byteStrToInt(string[20:28])/1E3
    heading = byteStrToInt(string[28:36])/1E5
    pitch = byteStrToInt(string[36:44])/1E5
    roll = byteStrToInt(string[44:52])/1E5
    turb = byteStrToInt(string[52:60])
    pdop = byteStrToInt(string[60:68])/1E2

    return [lat, lon, speed, heading, pitch, roll, turb, pdop]
    

In [4]:
def unpackBatteryVector(string):
    volts = byteStrToInt(string[4:12])/1E2
    amps = byteStrToInt(string[12:20])/1E2
    b2 = byteStrToInt(string[20:28])
    b3 = byteStrToInt(string[28:36])
    b4 = byteStrToInt(string[36:44])
    b5 = byteStrToInt(string[44:52])
    b6 = byteStrToInt(string[52:60])
    b7 = byteStrToInt(string[60:68])

    return [volts, amps, b2, b3, b4, b5, b6, b7]
    

In [5]:
def unpackControllerStateVector(string):
    steer = byteStrToInt(string[4:12])
    thrust = byteStrToInt(string[12:20])
    trim = byteStrToInt(string[20:28])
    error = byteStrToInt(string[28:36])

    return [steer, thrust, trim, error]
    

In [6]:
def unpackWaypoint(string):
    lat = byteStrToInt(string[4:12])/1E7
    lon = byteStrToInt(string[12:20])/1E7
    rad = byteStrToInt(string[20:28])
    type = byteStrToInt(string[28:36])
    ton = byteStrToInt(string[36:44])/1E3

    return [lat, lon, rad, type, ton]
    

In [7]:
def unpackWaypointList(string):
    list = []
    num_wps = int(string[4:6], 16)
    for i in range(num_wps):
        lat = byteStrToInt(string[(9*i+3)*2:(9*i+7)*2])/1E7
        lon = byteStrToInt(string[(9*i+7)*2:(9*i+11)*2])/1E7
        type = int(string[(9*i+8)*2:(9*i+8)*2+2], 16)
        list.append([lat, lon, type])

    return list
    

In [8]:
def unpackMissionParameters(string):
    curr_wp = byteStrToInt(string[4:12])
    num_wps = byteStrToInt(string[12:20])
    tac = byteStrToInt(string[20:28])
    cruise_speed = byteStrToInt(string[28:36])
    big_p_gain = byteStrToInt(string[36:44])
    small_p_gain = byteStrToInt(string[44:52])
    steer_i_gain = byteStrToInt(string[52:60])
    thrust_p_gain = byteStrToInt(string[60:68])
    trans_thresh = byteStrToInt(string[68:76])
    rough_sea = byteStrToInt(string[76:84])
    brownout_thresh = byteStrToInt(string[84:92])
    sat_update = byteStrToInt(string[92:100])
    exit_sf_md_time = byteStrToInt(string[100:108])
    lead_distance = byteStrToInt(string[108:116])

    return [curr_wp, num_wps, tac, cruise_speed, big_p_gain, small_p_gain, thrust_p_gain, trans_thresh, rough_sea, brownout_thresh, sat_update, exit_sf_md_time, lead_distance]
    

In [9]:
# call this to send a request a waypoint list message
def packRequestWPList():
    return "FFBBFE"
    
    

In [10]:
# call this to send a request a specific waypoint parameters message
def packRequestWP(num):
    return "FFBB" + str((hex(num))[2:]).zfill(2) + "FE"

    

In [11]:
def packChangeWaypoint(num, lat, lon, radius, type, tos):
    string = "FFBD"
    string+=str((hex(num))[2:]).zfill(2)
    string+=(binascii.hexlify((lat).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((lon).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((radius).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((type).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((tos).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=("FE")
    return string
    

In [12]:
# call this to send a request mission parameters message
def packGetMissionParameters():
    return "FFBEFE"
    

In [13]:
def packChangeMissionParameters(curr_wp, num_wps, tac, cruise_speed, big_p_gain, small_p_gain, steer_i_gain, thrust_p_gain, trans_thresh, rough_sea, brownout_thresh, sat_update, exit_sf_md_time, lead_distance):
    string = "FFBF"
    string+=(binascii.hexlify((curr_wp).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((num_wps).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((tac).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((cruise_speed).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((big_p_gain).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((small_p_gain).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((steer_i_gain).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((thrust_p_gain).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((trans_thresh).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((rough_sea).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((brownout_thresh).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((sat_update).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((exit_sf_md_time).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=(binascii.hexlify((lead_distance).to_bytes(4, signed=True, byteorder='little')).decode())
    string+=("FE")
    return string

In [14]:
# call this to send a force mode change message
def forceModeChange(num):
    return "FFCA" + str((hex(num))[2:]).zfill(2) + "FE"


In [15]:
def unpackGeneralUpdate(input):
    state_vector = unpackStateVector(input[0:34*2])
    battery_vector = unpackBatteryVector(input[34*2:68*2])
    control_vector = unpackControllerStateVector(input[68*2:(68+19)*2])
    gps1 = unpackStateVector(input[(68+19)*2:(102+19)*2])
    gps2 = unpackStateVector(input[(102+19)*2:(136+19)*2])
    wp = unpackWaypoint(input[(136+19)*2:(170+19)*2])

    print(state_vector)
    print(battery_vector)
    print(control_vector)
    print(gps1)
    print(gps2)
    print(wp)
    
    

In [16]:
# For a received message:
#   Unpack received message from a general update

input = "ffabd38a7017e93554d21000000087df5e01dbee0500deceffff000000007f000000ffab70090000bfffffff000000000000000000000000000000000000000000000000ffac9cffffff4b000000fdffffff3e57be00feffaa538b7017603654d2160000003f6e850134ae06003357feff0000000074000000ffaa548a7017713554d20a000000ca383801832f050089460100000000008c000000ffae1d8f65170aa468d23200000001000000a0860100fe07"
unpackGeneralUpdate(input)

[39.3251539, -76.6233111, 0.016, 229.94823, 3.88827, -0.12578, 0, 1.27]
[24.16, -0.65, 0, 0, 0, 0, 0, 0]
[-100, 75, -3, 12474174]
[39.3251667, -76.6232992, 0.022, 255.21727, 4.37812, -1.08749, 0, 1.16]
[39.3251412, -76.6233231, 0.01, 204.6177, 3.39843, 0.83593, 0, 1.4]
[39.2531741, -76.4894198, 50, 1, 100.0]


In [17]:
# For formatting a message to send:
#   Format a mission parameter update message

curr_wp = 0
num_wps = 1
tac = 0 # time at current
cruise_speed = 4000
big_p_gain = int(1E-5*1E6)
small_p_gain = int(1E-5*1E6)
steer_i_gain = int(1E-6*1E7)
thrust_p_gain = int(0.1*1E3)
trans_thresh = int(15E5)
rough_sea = 1000000
brownout_thresh = 2240
sat_update = 1800000
exit_sf_md_time = 0
lead_distance = 50

print(packChangeMissionParameters(curr_wp, num_wps, tac, cruise_speed, big_p_gain, small_p_gain, steer_i_gain, thrust_p_gain, trans_thresh, rough_sea, brownout_thresh, sat_update, exit_sf_md_time, lead_distance))

FFBF000000000100000000000000a00f00000a0000000a0000000a0000006400000060e3160040420f00c008000040771b000000000032000000FE


In [18]:
# For formatting a message to send:
#   Unpack a mission parameter receipt

print(unpackMissionParameters("FFBF000000000100000000000000a00f00000a0000000a0000000a0000006400000060e3160040420f00c008000040771b000000000032000000FE"))

[0, 1, 0, 4000, 10, 10, 100, 1500000, 1000000, 2240, 1800000, 0, 50]


In [19]:
# For a received message:
#   Unpack a waypoint list receipt

print(unpackWaypointList("fead0020ffffff7f00000000276a08002c7f082052000000a77608002c7f082003000000f9ffffff841a072042320900e8030000a776080003000000f9550800a6760800000000212c7f08200300000000000000915608002c7f0820395708002c7f08209f910000000000002c7f08200c7e0820b8020000137e0820030000009b91000000000000107e0820df5d080085310900107e08202c7f0820d3750000c37e08200c7e0820107e0820c37e0820b30000004b6008008a7d08208c7d08208e7d08203ea75d3f73c95701000037000100000088190720d13109007d73080065730800d1310900881907202c7f08200c7e0820ad730800f821072088190720"))

[]


In [23]:
# For formatting a message to send:
#   Format a waypoint update message

num = 1
lat = int(39.3254633*1E7)
lon = int(-76.6221929*1E7)
radius = 50
type = 0
tos = 1

print(packChangeWaypoint(num, lat, lon, radius, type, tos))

FFBD01e9967017976154d2320000000000000001000000FE


In [22]:
# For a received message:
#   Unpack a single waypoint receipt

print(unpackWaypoint("ffaee9967017976154d2320000000000000000000000fe"))

[39.3254633, -76.6221929, 50, 0, 0.0]
